# [Mars Lander](https://www.codingame.com/ide/puzzle/mars-lander)

My Ynote: "Coding Games - Mars Lander"<br>
[Study "Genetic Algorithm Implementation in Python" with helps of peforth](https://github.com/hcchengithub/GeneticAlgorithmPython/blob/master/Genetic%20Algorithm%20Implementation%20in%20Python.ipynb)

I didn't understand this game at first and that's a big concern to invest my time into developing the program. Thanks to [McKelian](https://www.codingame.com/forum/t/mars-the-game/756) and his [video game](http://www.mckelian.com/index.php/mars/choix) that mimics the Mars Lander puzzle.

<img src=marslander.png>
<img src=ControlPanel.png>

### Goal 
- land on flat ground
- land in a vertical position (tilt angle = 0°)
- vertical speed must be limited ( ≤ 40m/s in absolute value)
- horizontal speed must be limited ( ≤ 20m/s in absolute value)


# 調製 Simulator 成功了！

這套公式成功了。

Initialize

    thrust = 4    # the thrust command 
    rotate = 0    # the rotate command 
    a0 = 0        # absolute angle of the ship 
    pwr0 = 0      # 引擎出力
    gravity = 3.711  # constant on Mars
    p0 = v(2500, 2700)  # Initial position
    s0 = v(0,0)     # Initial speed vector
    g = angle2uv(-180) * gravity  # 重力向量
    step = 0
    
Run

    # f = m.a  因為 m 不變故當成 1 所以 f = a 也就是 force = g 
    # Duration t = 1 一步的時間都是 1 所以移動距離 s0 + (1/2)(force)(t^2) 變成 s0 + (1/2)*(force)
    
    step += 1

    a1 = angle15(a0,rotate)  # 這一步的角度 
    pwr1 = thrust_1(pwr0,thrust)  # 這一步的 引擎推力
    force = g + angle2uv(a1) * pwr1   # force on the ship during this step 
    s1 = s0 + force  # 這一步將達到的速度 s0 + (delta time)(acceleration) 
                     # where delta time is 1, a is force as mentioned above.
    p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置

    %f step -->
    %f a0 pwr0 -2 slice -->  # 上一步的 角度 與 引擎出力 0 1 2 3 4
    %f a1 pwr1 -2 slice -->  # 作用在這一步上的 角度 與 引擎出力
    %f force dup -->
    %f ===>      # 重力 與 引擎出力 的合力向量
    %f s1 ===>   # 這一步最後的速度
    %f p1 ===>   # 這一步最後的位置

    a0 = a1
    pwr0 = pwr1
    s0 = s1
    p0 = p1

# 寫出評分 function 

- 到達地表時的 位置、速度、角度、油料 決定得分。


# Laboratory

In [1]:
import peforth
peforth.bps = [i for i in range(100)]  # 預設有這麼多 Breakpoint ID
def bp(id=None,locals=None):
    # Breakpoint ID 不能超過 peforth.bps 保留，超過的無效。
    if id==None: 
        id = 0
        prompt='bp> '
    else:
        prompt="{}>".format(id)
    if id in peforth.bps: peforth.push(locals).ok(prompt, cmd="to _locals_")
peforth.bp = bp
peforth.push(bp).dictate("py: vm.bp=pop()")  # [ ] 忘了 vm.bp 有何用途？
peforth.dictate('''
    \ ------ breakpoint ------------------------------------------------------------
    : bp ." Usage: peforth.bp(11,locals())  # drop a breakpoint with ID=11" cr ;
    : bl // ( -- ) List all breakpoints
        __main__ :> peforth.bps 
        <py>
        bps = pop()
        print('Disabled breakpoints:')
        for i in range(len(bps)):
            if not bps[i]: 
                print(i, end=' ')
        print(); print('Enabled breakpoints:')
        count = 0
        for i in range(len(bps)):
            if bps[i]: 
                print(i, end=' ')
                count += 1
        print(); print('Enabled breakpoints count: {}/{}'.format(count,len(bps)))
        </py> cr ;
        /// Breakpoint commands:
        ///	  bl  - list all breakpoints (capital BL is white space) 
        ///	  be  - enable breakpoints, e.g. be 1 2 3 
        ///	  bd  - disable breakpoints, e.g. bd 1 2 3 
        ///	  be* - enable all breakpoints
        ///	  bd* - disable all breakpoints 
        /// Also: 
        ///   for i in [11,22,33]: peforth.bps[i]=0	 # disable breakpoints 11,22,33 
        ///   for i in [11,22,33]: peforth.bps[i]=i	 # enable  breakpoints 11,22,33 
    
    : bd // ( <1 2 3 4...> -- ) Disable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps )
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = 0
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : be // ( <1 2 3 4...> -- ) Enable listed breakpoints 
        CR word ( line ) __main__ :> peforth.bps ( line bps ) 
        <py>
        line, bps = pop(1), pop(0)
        points = map(int, line.split(' '))
        for i in points: bps[i] = i
        </py> ; 
        ' bl :> comment last :: comment=pop(1)
    
    : bd* // ( -- ) Disable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = 0
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    
    : be* // ( -- ) Enable all breakpoints 
        __main__ :> peforth.bps	 ( bps ) 
        <py>
        bps = pop()
        for i in range(len(bps)): bps[i] = i
        </py> ;
        ' bl :> comment last :: comment=pop(1)
    \ ------ xtack ------------------------------------------------------------
    [] value xstack xstack py: vm.xstack=pop() // ( -- array ) The xstack. 掛進 vm 就可以直接以 py> xstack 取用。
    : x@ xstack :> [-1] ; // ( -- n ) Get TOS of the xstack
    : x> xstack :> pop() ; // ( -- n ) Pop the xstack
    : >x xstack :: append(pop()) ; // ( n -- ) Push n into the xstack
    : .sx xstack . ; // ( -- ) List xstack 
    : xdrop x> drop ; // ( X: ... a -- X: ... ) drop xstack 
    : xdropall [] to xstack ; // ( X: ... -- X: empty ) clear xstack 
    ''')
%f version ==>


reDef unknown
reDef -->
p e f o r t h    v1.24
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.

version ==>
 1.24 (<class 'str'>)


In [2]:
import numpy as np
import re, math, random, pdb, json 
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

p e f o r t h    v1.24
source code http://github.com/hcchengithub/peforth
Type 'peforth.ok()' to enter forth interpreter, 'exit' to come back.



### Read log file 

In [40]:
# Method 1, from another notebook
%store -r log
log

[{'step': 0,
  'length': 91,
  'x': 2500,
  'y': 2500,
  'hspeed': 0,
  'vspeed': 0,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (-99, -99)},
 {'step': 1,
  'length': 91,
  'x': 2500,
  'y': 2498,
  'hspeed': 0,
  'vspeed': -4,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 2,
  'length': 91,
  'x': 2500,
  'y': 2493,
  'hspeed': 0,
  'vspeed': -7,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 3,
  'length': 91,
  'x': 2500,
  'y': 2483,
  'hspeed': 0,
  'vspeed': -11,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 4,
  'length': 91,
  'x': 2500,
  'y': 2470,
  'hspeed': 0,
  'vspeed': -15,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 5,
  'length': 91,
  'x': 2500,
  'y': 2454,
  'hspeed': 0,
  'vspeed': -19,
  'fuel': 500,
  'angle': 0,
  'power': 0,
  'cmd': (0, 0)},
 {'step': 6,
  'length': 91,
  'x': 2500,
  'y': 2433,
  'hspeed': 0,
  'vspeed': -22,
  'fuel': 500,
  'angle':

In [5]:
# Method 2, Read back the json file
%f \ s" actual free fall.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
%f s" all power.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
log = peforth.execute('log').pop()

In [123]:
gravity = -3.711        # constant on Mars
g = gravity             # 到了二維以上，重力需轉成向量

class State:

    def __init__(self,step=0,x=0,y=0,hspeed=0,
            vspeed=0,fuel=0,angle=0,power=0,force=0):
        self.step   = step
        self.x      = x
        self.y      = y
        self.hspeed = hspeed
        self.vspeed = vspeed
        self.fuel   = fuel
        self.angle  = angle
        self.power  = power
        self.force  = force

    def next(self,angle,power):  
        # input angle,power 都是 offset 分別屬 -15~15 以及 -1,0,+1 
        assert power in [-1, 0, 1], 'expected input power is an offset [-1, 0, +1], given {}'.format(power)
        assert angle in range(-15,16), 'input angle range [-15:16], given {}'.format(angle)
        self.step   += 1
        self.angle  += angle
        self.angle   = -90 if self.angle < -90 else self.angle
        self.angle   =  90 if self.angle > 90 else self.angle
        self.power  += power
        self.power   = 0 if self.power < 0 else self.power
        self.power   = 4 if self.power > 4 else self.power
        self.fuel   -= self.power
        self.x       = self.x       # episode-1 不變
        self.hspeed  = self.hspeed  # episode-1 不變
        self.force   = g + self.power
        self.y       = self.y + self.vspeed + (1/2)*self.force
        self.vspeed += self.force
        return self  # 傳回值只是方便測試，可以這樣寫： %f state :> next(0,3.711) -->

    def __str__(self):  
        return str(self.__dict__)

# 測試兼範例
# 模擬自由落體
state = State(0,0,2500,0,0,500,0,0,0)
%f 36 [for] state :> next(0,0) [next] 
%f ( Free fall 墜毀 at step 36 ) state --> cr 
# 模擬安全降落
state = State(0,0,2500,0,0,500,0,0,0)
%f 9 [for] state :> next(0,0) [next] 
%f 91 9 - [for] state :> next(0,1) [next] 
%f ( 9 步以後狂噴到 91 步安全著陸 ) state --> cr 
# 模擬飛向太空
state = State(0,0,2500,0,0,500,0,0,0)
%f 83 [for] state :> next(0,1) [next] 
%f ( 83 步被判逸失 ) state -->

( Free fall 墜毀 at step 36 ) state --> {'step': 36, 'x': 0, 'y': 95.2720000000003, 'hspeed': 0, 'vspeed': -133.59599999999998, 'fuel': 500, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)

( 9 步以後狂噴到 91 步安全著陸 ) state --> {'step': 91, 'x': 0, 'y': 97.6045000000063, 'hspeed': 0, 'vspeed': -15.700999999999876, 'fuel': 178, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)

( 83 步被判逸失 ) state --> {'step': 83, 'x': 0, 'y': 3004.4604999999906, 'hspeed': 0, 'vspeed': 17.987000000000002, 'fuel': 174, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)


In [126]:
# 模擬飛向太空
state = State(0,0,2500,0,0,500,0,0,0)
%f 83 [for] state :> next(0,1) [next] 
%f ( 83 步被判逸失 ) state -->
score = None  # 有可能是負的，先給個看得出異常的初值。

# 先判斷是否觸地。若觸地進一步查看速度合格是否合格。若未觸地就是飛走了或油料耗盡。
if state.y <= 100:
    # 觸地了，進一步查看速度是否合格
    if state.vspeed >= -40 : # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小
        # 安全降落
        score = state.fuel
    else:  # 墜毀
        # 模擬噴火把速度降到合格範圍內。所剩油料就是成績。
        while True:
            state.next(0,1)
            if state.vspeed >= -40:
                score = state.fuel
                break
else:  # 還在空中，飛走了或油料耗盡。
    # 先模擬 free fall 觸地。
    while True: 
        state.next(0,-1)  # 半空中開始模擬 free fall 小心別又錯了，不是直接填 0 ！！
        if state.y <= 100: break
    # 然後噴火把速度降到合格範圍內，所剩油料就是成績。
    while True:
        if state.vspeed >= -40: # 剛從空中下來觸地， vspeed 一定是負值；墜毀的一定比 -40 小。
            # 達標了
            score = state.fuel
            break
        else: # 未達標，繼續噴火把負的 vspeed 速度降到合格範圍內
            state.next(0,1)
    
%f score -->



( 83 步被判逸失 ) state --> {'step': 83, 'x': 0, 'y': 3004.4604999999906, 'hspeed': 0, 'vspeed': 17.987000000000002, 'fuel': 174, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)
score --> -1438 (<class 'int'>)


### Free Fall simulatin is ok 

In [5]:
# Read log json file
%f s" actual free fall.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
log = peforth.execute('log').pop()

In [28]:
# Initial state of the game
# Run once for a specific experiment e.g. free fall because Log files are different for each experiment.
log0 = log[0].copy();
del log0['length'];  # 剃除 web 上 log 到多的東西
del log0['cmd']      # 剃除 web 上 log 到多的東西
state0 = State(**log0)  # 取 log 轉成 state object 
%f log :> [0] -->
%f state0 -->

NameError: name 'log' is not defined

In [1316]:
# Initialize the free fall simulation 
state = State(**state0.__dict__)  # Initial state of the game

# Read log file for comparison
%f s" actual free fall.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
log = peforth.execute('log').pop()

reDef log


In [1352]:
# Ctrl-Enter repeatedly to compare state of each step with the log 
state.next(0,0)  # 自由落體 free fall power 總是 0  
%f state :> __dict__.keys() list . cr
_ = map(lambda x: int(round(x,0)), state.__dict__.values())  # 四捨五入
%f _ list . cr 
print(list(log[state.step].keys()))
print(list(log[state.step].values()))  # 對照 log 到的 data

['step', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'force']
[36, 2500, 95, 0, -134, 500, 0, 0, -4]
['step', 'length', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'cmd']
[36, 36, 2500, 100, 0, -134, 500, 0, 0, [0, 0]]


In [1353]:
# 保留 free fall 最後結果方便驗算 scoring function 
state_free_fall = State(**state.__dict__)  # copy the last state from the end of the free fall

In [1354]:
# Scoring and get the cost
%f state_free_fall :> y -->
%f ( 觸地了嗎？ ) state_free_fall :> y 100 <= -->
%f state_free_fall :> vspeed -->
%f ( 安全著陸嗎？ ) state_free_fall :> vspeed abs 40 <= -->
%f ( 若安全著陸，所剩油料就是分數 ) state_free_fall :> fuel -->
%f ( 觸地時的 power ) state_free_fall :> power -->
state_scoring = State(**state_free_fall.__dict__)
%f cr ( 察覺以上結果失敗，準備按墜毀方式計算分數 ) state_scoring -->

state_free_fall :> y --> 95.2720000000003 (<class 'float'>)
( 觸地了嗎？ ) state_free_fall :> y 100 <= --> True (<class 'bool'>)
state_free_fall :> vspeed --> -133.59599999999998 (<class 'float'>)
( 安全著陸嗎？ ) state_free_fall :> vspeed abs 40 <= --> False (<class 'bool'>)
( 若安全著陸，所剩油料就是分數 ) state_free_fall :> fuel --> 500 (<class 'int'>)
( 觸地時的 power ) state_free_fall :> power --> 0 (<class 'int'>)

cr ( 察覺以上結果失敗，準備按墜毀方式計算分數 ) state_scoring --> {'step': 36, 'x': 2500, 'y': 95.2720000000003, 'hspeed': 0, 'vspeed': -133.59599999999998, 'fuel': 500, 'angle': 0, 'power': 0, 'force': -3.711} (<class '__main__.State'>)


In [1697]:
# 墜毀時的速度減到+-40 以內需要多少油料？ 從當時的 power 開始要用力噴火減速。
# 修正自由落體墜毀需要的油料繼續從 fuel 裡扣除，
# 最後修正到 vspeed = -40 時 step = 379, fuel = -866 此即這個 genome 的分數了。

power = min(state_scoring.power + 1, 4)
state_scoring.next(0,power)
%f state_scoring :> __dict__.keys() list . cr
_ = map(lambda x: int(round(x,0)), state_scoring.__dict__.values())
%f _ list . cr 


['step', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'force']
[379, 2500, -30779, 0, -40, -866, 0, 4, 0]


## Free fall 9 steps and then thrust with all power
這是個會安全著陸的 genome.

In [1698]:
# Initialize this simulation 
state = State(**state0.__dict__)  # copy the last state from the end of the free fall

# Read log file for comparison
%f s" 先掉10步然後狂噴.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
log = peforth.execute('log').pop()

reDef log


In [1789]:
# Ctrl-Enter repeatedly to compare state of each step with the log 
power = 0 if state.step < 9 else min(state.power+1,4)  # 本實驗專用 
state.next(0,power)  # Free fall 9 steps and then thrust with all power
%f state :> __dict__.keys() list . cr
_ = map(lambda x: int(round(x,0)), state.__dict__.values())
%f _ list . cr 
print(list(log[state.step].keys()))
print(list(log[state.step].values()))

['step', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'force']
[91, 2500, 98, 0, -16, 178, 0, 4, 0]
['step', 'length', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'cmd']
[91, 91, 2500, 100, 0, -16, 178, 0, 4, [0, 4]]


In [1790]:
# 保留最後結果方便驗算 scoring function 
state_ok = State(**state.__dict__)  # copy the last state

In [1792]:
# Scoring and get the cost
%f state_ok :> y -->
%f ( 觸地了嗎？ ) state_ok :> y 100 <= -->
%f state_ok :> vspeed -->
%f ( 安全著陸嗎？ ) state_ok :> vspeed abs 40 <= -->
%f ( 若安全著陸，所剩油料就是分數 ) state_ok :> fuel -->
%f ( 觸地時的 power ) state_ok :> power -->
%f cr ( 察覺以上結果成功 ，無須另外計算分數 ) "" -->

state_ok :> y --> 97.6045000000063 (<class 'float'>)
( 觸地了嗎？ ) state_ok :> y 100 <= --> True (<class 'bool'>)
state_ok :> vspeed --> -15.700999999999876 (<class 'float'>)
( 安全著陸嗎？ ) state_ok :> vspeed abs 40 <= --> True (<class 'bool'>)
( 若安全著陸，所剩油料就是分數 ) state_ok :> fuel --> 178 (<class 'int'>)
( 觸地時的 power ) state_ok :> power --> 4 (<class 'int'>)

cr ( 察覺以上結果成功 ，無須另外計算分數 ) "" -->  (<class 'str'>)


## Thrust with all power 直到油料耗盡

油料耗盡時的高度距離與速度以自由落體觸地 (期間沒有耗用油料)，然後把速度調整到+-40 以內需要多少油料？這時候從空中 free fall 下來的 vspeed 一定是負的，只有 -40 以下才需要噴火調整，故只有一個方向 (不用考慮有另一方向也需要調整的可能)。

從頭到尾全速噴火直到油料耗盡 web 端的程式很簡單，只改一行：

    # 2 integers: rotate power. rotate is the desired ......
    print("0 4")  # 0 0 改成 0 4 

結果 the lander 飛走了 (y > 3000) 而被判 failed. Log 檔 "all power.json". 這組 log 有高度、速度、推力、油料等數據，照樣可以用來練習油料耗盡的狀況，因此時油料並未耗盡，故反而更周延。

In [3]:
# Read log json file
%f s" all power.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
log = peforth.execute('log').pop()

In [27]:
# Initial state of the game
# Run once for a specific experiment e.g. free fall because Log files are different for each experiment.
log0 = log[0].copy();
del log0['length'];  # 剃除 web 上 log 到多的東西
del log0['cmd']      # 剃除 web 上 log 到多的東西
state0 = State(**log0)  # 取 log 轉成 state object 
%f log :> [0] -->
%f state0 -->

NameError: name 'log' is not defined

In [7]:
# Initialize this simulation 
state = State(**state0.__dict__)  # copy the initial state of the game

# Read the log json file
%f s" all power.json" readTextFile py> eval(pop()) constant log // ( -- json ) "actual free fall.json" log from Codingame > Mars Lander > episode-1 
log = peforth.execute('log').pop()

reDef log


In [90]:
# Ctrl-Enter repeatedly to compare state of each step with the log 
power = min(state.power+1,4)  # all power 實驗專用。這裡一開始搞錯了！不是直接放 4 !!!
state.next(0,power) 
%f state :> __dict__.keys() list . cr
_ = map(lambda x: int(round(x,0)), state.__dict__.values())
%f _ list . cr 
print(list(log[state.step].keys()))
print(list(log[state.step].values()))

['step', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'force']
[83, 2500, 3004, 0, 18, 174, 0, 4, 0]
['step', 'length', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'cmd']
[83, 83, 2500, 3004, 0, 18, 174, 0, 4, [0, 4]]


In [91]:
# 保留最後結果方便驗算 scoring function 
state_all_power = State(**state.__dict__)  # copy the last state

In [190]:
# Scoring and get the cost
%f state_all_power :> y -->
%f ( 觸地了嗎？ ) state_all_power :> y 100 <= -->
%f state_all_power :> vspeed -->
%f ( 安全著陸嗎？ ) state_all_power :> vspeed abs 40 <= -->
%f ( 若安全著陸，所剩油料就是分數 ) state_all_power :> fuel -->
%f ( 觸地時的 power ) state_all_power :> power -->
state_scoring = State(**state_all_power.__dict__)
%f cr ( 察覺以上結果失敗，準備按墜毀方式計算分數 ) state_scoring -->

state_all_power :> y --> 3004.4604999999906 (<class 'float'>)
( 觸地了嗎？ ) state_all_power :> y 100 <= --> False (<class 'bool'>)
state_all_power :> vspeed --> 17.987000000000002 (<class 'float'>)
( 安全著陸嗎？ ) state_all_power :> vspeed abs 40 <= --> True (<class 'bool'>)
( 若安全著陸，所剩油料就是分數 ) state_all_power :> fuel --> 174 (<class 'int'>)
( 觸地時的 power ) state_all_power :> power --> 4 (<class 'int'>)

cr ( 察覺以上結果失敗，準備按墜毀方式計算分數 ) state_scoring --> {'step': 83, 'x': 2500, 'y': 3004.4604999999906, 'hspeed': 0, 'vspeed': 17.987000000000002, 'fuel': 174, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)


In [891]:
state = State(**state_all_power.__dict__)  #

In [892]:
%f state :> __dict__ -->

state :> __dict__ --> {'step': 83, 'x': 2500, 'y': 3004.4604999999906, 'hspeed': 0, 'vspeed': 17.987000000000002, 'fuel': 174, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class 'dict'>)


In [937]:
# Ctrl-Enter repeatedly to compare state of each step with the log 
# 小心！ 又錯了，不是直接填 0 ！！ state.next(0,0)  # 自由落體 free fall power 總是 0  
%f state :> __dict__.keys() list . cr
_ = map(lambda x: int(round(x,0)), state.__dict__.values())  # 四捨五入
%f _ list . cr 
# print(list(log[state.step].keys()))
# print(list(log[state.step].values()))  # 對照 log 到的 data

['step', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'force']
[128, 2500, 56, 0, -149, 174, 0, 0, -4]


In [441]:
state_scoring = State(**state.__dict__)  #

In [841]:
power = min(state_scoring.power + 1, 4)
state_scoring.next(0,power)
%f state_scoring :> __dict__.keys() list . cr
_ = map(lambda x: int(round(x,0)), state_scoring.__dict__.values())
%f _ list . cr 


['step', 'x', 'y', 'hspeed', 'vspeed', 'fuel', 'angle', 'power', 'force']
[528, 2500, -38820, 0, -39, -1420, 0, 4, 0]


In [1321]:
# Run once for initial state 
gravity = -3.711        # constant on Mars
g = gravity            # 重力轉成向量
step = 0 
gene = log[step]
p0 = gene['y']
s0 = gene['vspeed']
fuel0 = gene['fuel']
a0 = gene['angle']    # 這一步的角度 
powr0 = gene['power']  # 這一步的 引擎推力
cmd0 = (0,0)
state0 = (step ,p0 ,s0 ,fuel0 ,a0 ,powr0 ,cmd0)
log0 = (log[0]['step'] ,log[0]['y'] ,log[0]['vspeed'] ,log[0]['fuel'] ,log[0]['angle'] ,log[0]['power'] ,log[0]['cmd'])

%f log :> [0] -->
%f state0 -->
%f log0 -->


log :> [0] --> {'step': 0, 'length': 83, 'x': 2500, 'y': 2500, 'hspeed': 0, 'vspeed': 0, 'fuel': 500, 'angle': 0, 'power': 0, 'cmd': [-99, -99]} (<class 'dict'>)
state0 --> (0, 2500, 0, 500, 0, 0, (0, 0)) (<class 'tuple'>)
log0 --> (0, 2500, 0, 500, 0, 0, [-99, -99]) (<class 'tuple'>)


In [1404]:
# run multiple times to see if the simulation match with the log, it does!!
step += 1
a1 = 0
powr1 = min(powr0+1,4)  # all power 實驗專用。這裡一開始搞錯了！不是直接放 4 !!!
fuel1 = fuel0 - powr1
force = g + powr1   # force on the lander ship during this step 
s1 = s0 + force  # 這一步將達到的速度 s0 + (delta time)(acceleration)  # where delta time is 1, a is force as mentioned above.
p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel1 ,a1 ,powr1 ,(a1,powr1))

p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
try:
    log1 = (step ,log[step]['y'] ,log[step]['vspeed'] ,log[step]['fuel'] ,log[step]['angle'] ,log[step]['power'] ,log[step]['cmd'])
except:
    pass
%f ( log-length ) log :> [0]['length'] -->
%f state1 -->
%f log1 -->

# 結果到了 step 83 lander 高度超過 3000 被判定為飛走了 
# ( log-length ) log :> [0]['length'] --> 83 (<class 'int'>)
# state1 --> (83, 3004, 18, 174, 0, 4, (0, 4)) (<class 'tuple'>)
# log1 --> (83, 3004, 18, 174, 0, 4, [0, 4]) (<class 'tuple'>)

( log-length ) log :> [0]['length'] --> 83 (<class 'int'>)
state1 --> (83, 3004, 18, 174, 0, 4, (0, 4)) (<class 'tuple'>)
log1 --> (83, 3004, 18, 174, 0, 4, [0, 4]) (<class 'tuple'>)


In [1459]:
# Initialize 
# 從被判 fail 的狀態開始模擬自由落體到觸地為止
step, p0, s0, fuel0, powr0 = log1[0], log1[1], log1[2], log1[3], log1[5]
state0 = (step ,p0 ,s0 ,fuel0 ,log1[4] ,powr0 )
%f log1 -->
%f state0 -->

log1 --> (83, 3004, 18, 174, 0, 4, [0, 4]) (<class 'tuple'>)
state0 --> (83, 3004, 18, 174, 0, 4) (<class 'tuple'>)


In [1506]:
# run multiple times to simulate free fall 
step += 1
a1 = 0
powr1 = max(powr0-1,0)  # 半途開始模擬 free fall 
fuel1 = fuel0 - powr1
force = g + powr1 
s1 = s0 + force 
p1 = p0 + s0 + (1/2)*(force)
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel1 ,a1 ,powr1 ,(a1,powr1))
p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
%f state1 -->

# free fall 到 step 130 觸地，這時 vspeed -150 準備用來換算成修正所需的油料。
# state1 --> (130, 26, -150, 168, 0, 0, (0, 0)) (<class 'tuple'>)

state1 --> (130, 26, -150, 168, 0, 0, (0, 0)) (<class 'tuple'>)


In [1507]:
# Scoring and get the cost
%f p1 -->
%f ( 觸地了嗎？ ) p1 100 <= -->
%f s1 -->
%f ( 安全著陸嗎？ ) s1 abs 40 <= -->
%f ( 若安全著陸，所剩油料就是分數 ) fuel1 -->
%f ( 觸地時的 power ) powr0 -->

p1 --> 26.200499999998563 (<class 'float'>)
( 觸地了嗎？ ) p1 100 <= --> True (<class 'bool'>)
s1 --> -150.41700000000003 (<class 'float'>)
( 安全著陸嗎？ ) s1 abs 40 <= --> False (<class 'bool'>)
( 若安全著陸，所剩油料就是分數 ) fuel1 --> 168 (<class 'int'>)
( 觸地時的 power ) powr0 --> 0 (<class 'int'>)


In [1910]:
# 墜毀時的速度減到 +-40 以內需要多少油料？ 從當時的 power 開始要用力噴火減速。
# 結果發現，修正自由落體墜毀需要的油料繼續從 fuel 裡扣除，
# 最後修正到 vspeed = -40 時 fuel = -1438 此即這個 genome 的分數了。
%f ( 前 ) state1 -->
step += 1
a1 = 0
powr1 = min(powr0 + 1, 4)
fuel1 = fuel0 - powr1
force = g + powr1
s1 = s0 + force
p1 = p0 + s0 + (1/2)*(force)  # where "s0 + (1/2)*(force)" is position delta 這一步將到達的位置
state1 = (step ,int(round(p1,0)) ,int(round(s1,0)) ,fuel1 ,a1 ,powr1 ,(a1,powr1))
p0, s0, fuel0, powr0 = p1, s1, fuel1, powr1
%f ( 後 ) state1 -->

# 最終把 vspeed 調整回到 -40 所剩油料是 -1438
# ( 前 ) state1 --> (531, -39454, -41, -1430, 0, 4, (0, 4)) (<class 'tuple'>)
# ( 後 ) state1 --> (532, -39495, -40, -1434, 0, 4, (0, 4)) (<class 'tuple'>)
# 以上是 round-off error, 再下一步 step 533 才是真的在 +-40之內，fuel -1438 才對。

( 前 ) state1 --> (531, -39454, -41, -1430, 0, 4, (0, 4)) (<class 'tuple'>)
( 後 ) state1 --> (532, -39495, -40, -1434, 0, 4, (0, 4)) (<class 'tuple'>)


# 筆記


2021.4.24 05:43 Scoring 要有一套客觀的計算方式. I have a much better idea now, finally. See [OneNote2021 >> Mars Lander - Episode 1](https://onedrive.live.com/view.aspx?resid=A796EA18AC8C1DA9%2115651&id=documents&wd=target%28Genetic%20Algorithm.one%7C3D994361-3562-4730-99E4-DAC83DAD73F2%2FMars%20Lander%20-%20Episode%201%7C156EADBB-D6A0-4C92-8BDD-043C48CC8271%2F%29)
# 寫出評分 function 

- 到達地表時的 位置、速度、角度、油料 決定得分。
- 與平地的距離要越近越好。

`[X]` 點到線的距離 `distance = np.cross(p2-p1,p3-p1)/np.linalg.norm(p2-p1)` where p2-p1 is the line and p3 is the point. 

`[ ]` 判斷著地的方法就很簡單了，用 ship 的位置來算與每個 surface 的距離，如果到了平地就是降落了，如果到了其他 surface 就是撞毀了。

`[ ]` 降落平地還要看角度(0)、速度(20,40)，不對還是失敗。

`[ ]` 降落平地之後，油料用得少決定勝負。

1. Crashed 或 landed 得分都是與 flate ground 的直線距離 (越近越好) 
   與 B0 = np.linalg.norm(v(7000,3000)) 的差距，最高分就是這個值。

2. landed 還有加分 bonus, 角度越接近 0 越好，Bonus 要比前一個分數高，使得 crash 完全沒得比。
   角度是 -90~90 之間，故 +-90度得 B0 分，而 0度得 2B0 分，推導出: <br>
    `bonus_angle(an) = 2B0 - (B0/90)*abs(an)`

3. 水平速度要 20 以下，垂直速度要 40 以下。同上，水平速度 500 時得 B0 分，而 20 以下時得 2B0 分<br>
    `bonus_sh(sh) = 2B0 - (B0/500)*abs(0 if sh<20 else sh)`<br>
    `bonus_sv(sv) = 2B0 - (B0/500)*abs(0 if sv<40 else sv)`

4. 如果三個都滿足，再加上 fuel 看剩多少。都用完了得 B0 分，都沒用得 2B0 分。

    `bonus_fuel(fuel) = B0 + (B0/Fuel)*(fuel)`


### 做出一個 class 叫作 Ship 
有油料、位置、速度、角度、引擎出力、軌跡、從開始到落地的一長串 chromosome 也就是一長串的基因 genes (rotate,power) 、還有 score。

先亂撒一組命令 > get the distance if alives then repeat else if crash then record the final distance (to the flate ground) as its score. 嘗試的過程不評分，要撞毀了或落地了才評分。
如果 parent 都撞毀了，誰好? 簡化地形為 surface 一律朝上，則越接近 flate ground 的就是越好。若有凹洞, 也就是 surface 有朝下時，要另外想辦法。如果 parent 都撞毀了，但是其子代 gene 都用完了還 inTheAir 就要自己用亂數探索自己的前途，最終要撞毀了或落地了才評分這個子代。經過 crossover and mutation 或單純的 exploration 都一定會跑出雜枝，不影響評分與演化過程因為會被 ship.move() 修正好。


## 用 Genetic Algorithm 先找出一條成功的路徑

一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 100 或 1K 或 1W 名烏合之眾的前 15 名，兩兩交配湊 105 個下一代，再取前 15 名兩兩配對又得 105 個子代。如此輾轉，看看結果。<br>
一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。<br>

In [ ]:
# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        p0 = v(2500, 2700)  # Initial position
        a0 = 0        # absolute angle of the ship 
        pwr0 = 0      # 引擎出力
        s0 = v(0,0)     # Initial speed vector
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break

In [ ]:
epoch = 0 

In [ ]:
# mating 用 parents 排列組合出下一代
spouses = [c for c in combinations(parents, 2)]

In [ ]:
# crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
offspring = []
for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
    offspring_crossover = crossover(spouses[i])
    if len(offspring_crossover) >= epoch :
        # 不一定 mutate 得到，看長度夠不夠
        offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
    offspring.append([0,offspring_crossover])  # _working_

In [ ]:
# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 手動連續跌代 (舊版，全部 gene 一起 mutate)

### 手動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
# mating 用 parents 排列組合出成對的配偶
spouses = [c for c in combinations(parents, 2)]

# crossover 產生下一代，隨即令其突變
offspring = []
for i in range(len(spouses)):
    offspring.append([0,list(map(mutation,crossover(spouses[i])))])

# evaluate the offspring populataion
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
for i in range(len(offspring)):
    offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200
    ship = Ship(p0, s0, a0, pwr0, fuel0)
    ship.evaluate(offspring[i][1])
    offspring[i][0] = ship.score
    %f i ship :> score -2 slice -->
epoch += 1 
# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 自動連續跌代

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

In [ ]:
%run mars.py

# Get population 
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        p0 = v(2500, 2700)  # Initial position
        a0 = 0        # absolute angle of the ship 
        pwr0 = 0      # 引擎出力
        s0 = v(0,0)     # Initial speed vector
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break
# 開始自動演化
high0 = 0
for j in range(1000):
    # mating 用 parents 排列組合出成對的配偶
    spouses = [c for c in combinations(parents, 2)]

    # crossover 產生下一代，隨即令其突變
    offspring = []
    for i in range(len(spouses)):
        offspring.append([0,list(map(mutation,crossover(spouses[i])))])

    # evaluate the offspring populataion
    p0 = v(2500, 2700)  # Initial position
    a0 = 0        # absolute angle of the ship 
    pwr0 = 0      # 引擎出力
    s0 = v(0,0)     # Initial speed vector
    for i in range(len(offspring)):
        offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.evaluate(offspring[i][1])
        offspring[i][0] = ship.score

    # 上下兩代組合成新一輪的 parents 
    parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
    high1 = [row[0] for row in parents][4]
    if j%10 == 0 : print(f"#{j}", end=" ") 
    if high0 != high1 : print(f"{j}:{high1}")
    high0 = high1

# evaluate a specific chromosome
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(parents[4][1])
%f ship :> make_score() ship :> score -->
%f ship :> chromosome -->


# 修正 mutation 的方式
_working_
2019/06/13 17:41 Mutation 的方式不好，咱目前是一次所有的 gene 全部 mutate，可能是造成訓練效果不佳的原因！因為某個 gene 的 mutation 再怎麼好效果被沖淡了，無法彰顯。所以要一組一組從頭照順序來讓 Mutation 找出最佳組合。這樣看來，窮舉是 93 種組合 for 一組 gene (-15~15, -1~1)，我目前是 10 個 offspring 只有 1/9, 將來考慮改多點。窮舉可能不見得好，因為它假設後面的一長串都是對的才有意義，當然這大有疑問，所以要大家一起慢慢演化才好。從頭 mutation 到尾 (每條 chromosome 的長度不一樣，看哪一條最長都要涵蓋到) 就是一輪 epoch. 

### 手動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop

In [ ]:
%run mars.py

In [ ]:
epoch = 0 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector

In [ ]:
# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break

In [ ]:
# mating 用 parents 交配出親代的排列組合
spouses = [c for c in combinations(parents, 2)]

# crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
offspring = []
for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
    offspring_crossover = crossover(spouses[i])
    if len(offspring_crossover) >= epoch :
        # 不一定 mutate 得到，看長度夠不夠
        offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
    offspring.append([0,offspring_crossover])

offspring_length = len(offspring)
epoch_end = 0 # 如果 epoch count 大過所有的 chromosome_length 則 epoch 就該結束了
for i in range(offspring_length):
    offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200  # 重複最後一組 gene 保證使 ship 落地
    ship = Ship(p0, s0, a0, pwr0, fuel0)
    chromosome_length = ship.evaluate(offspring[i][1])
    if epoch > chromosome_length : epoch_end += 1
    offspring[i][0] = ship.score
    %f i ship :> score -2 slice -->
        
if epoch_end >= offspring_length :
    %f ." End of epoch"
    epoch = 0
else:
    epoch += 1
    %f epoch --> 



# 上下兩代組合成新一輪的 parents 
parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
[row[0] for row in parents]    

### 自動連續跌代 (新版，個別 gene 依序逐個 mutate)

In [ ]:
import numpy as np
import re, math, peforth, random, pdb
from itertools import combinations  # combinations 數學「排列組合」中的「組合」
%f version drop
%run mars.py

# constants of the game world 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector

# Get population
# 一見面先取 5 名烏合之眾兩兩交配湊 10 個下一代，再取前 5 名兩兩配對又得 10 個子代。如此輾轉，看看結果。
parents = []
for i in range(5):
    while True:
        ship = Ship(p0, s0, a0, pwr0, fuel0)
        ship.explore()
        if ship.score > 0 : 
            parents.append((ship.score, ship.chromosome))
            break
high0 = 0            
for cycle in range(1000):
    epoch = 0  # a new cycle 
    while(True):
        # mating 用 parents 交配出親代的排列組合
        spouses = [c for c in combinations(parents, 2)]

        # crossover 產生下一代，隨即令其突變 (從頭逐一 gene 依序突變)
        offspring = []
        for i in range(len(spouses)):  # 10 個子代都針對某一組 (epoch) gene 突變，大家來較量
            offspring_crossover = crossover(spouses[i])
            if len(offspring_crossover) >= epoch :
                # 不一定 mutate 得到，看長度夠不夠
                offspring_crossover[epoch] = mutation(offspring_crossover[epoch])  # 只 mutate 一個 gene
            offspring.append([0,offspring_crossover])

        # 為這些子代評分
        offspring_length = len(offspring)
        epoch_end = 0 # 如果 epoch count 大過所有的 chromosome_length 則 epoch 就該結束了
        for i in range(offspring_length):
            offspring[i][1] = offspring[i][1] + [offspring[i][1][-1]]*200  # 重複最後一組 gene 保證使 ship 落地
            ship = Ship(p0, s0, a0, pwr0, fuel0)
            chromosome_length = ship.evaluate(offspring[i][1])
            if epoch > chromosome_length : epoch_end += 1
            offspring[i][0] = ship.score
                
        # 上下兩代組合成新一輪的 parents 
        parents = sorted(offspring + parents, key=lambda x:x[0])[-5:]
        high1 = parents[4][0]
        if high0 != high1 :
            print(f"cycle:{cycle} epoch:{epoch} highest score:{high1}") 
            high0 = high1

        if epoch_end >= offspring_length :
            print(f"cycle:{cycle} ~~~~ End of epoch ~~~~")
            break
        else:
            epoch += 1

In [ ]:
parents[4][0]

# evaluate a specific chromosome

In [6]:
# evaluate a specific chromosome
chromosome = [[-15, 0], [0, 1], [-1, 2], [6, 1], [20, 0], [5, 1], [17, 0], [2, 1], [-12, 2], [-1, 3], [14, 4], [8, 4], [-3, 4], [-14, 3], [-26, 3], [-11, 3], [0, 4], [4, 3], [-11, 3], [-5, 4], [10, 3], [8, 2], [-7, 1], [-14, 2], [-20, 3], [-22, 4], [-7, 3], [-17, 4], [-17, 4], [-14, 3], [-29, 4], [-44, 4], [-35, 3], [-20, 4], [-35, 4], [-48, 4], [-46, 4], [-42, 4], [-27, 4], [-15, 4], [-19, 3], [-22, 4], [-25, 4], [-25, 3], [-19, 3], [-26, 3], [-24, 4], [-34, 4], [-49, 4], [-34, 3], [-49, 4], [-34, 3], [-49, 4], [-42, 4], [-33, 4], [-41, 3], [-42, 3], [-41, 2], [-26, 3], [-11, 2], [4, 1], [-11, 0], [-1, 1], [-15, 0], [0, 1],[0,0]] 
p0 = v(2500, 2700)  # Initial position
a0 = 0        # absolute angle of the ship 
pwr0 = 0      # 引擎出力
s0 = v(0,0)     # Initial speed vector
ship = Ship(p0, s0, a0, pwr0, fuel0)
ship.evaluate(chromosome)
%f ship :> score -->
%f ship :> chromosome -->
%f ship :> inTheAir -->
%f ship :> p0 -->     
trace = list(map(lambda x:np.round(x,0), ship.trace))
%f trace -->
%f ship :> trace -->



ship :> score --> 63531.77246565923 (<class 'numpy.float64'>)
ship :> chromosome --> [[-15, 0], [0, 1], [-1, 2], [6, 1], [20, 0], [5, 1], [17, 0], [2, 1], [-12, 2], [-1, 3], [14, 4], [8, 4], [-3, 4], [-14, 3], [-26, 3], [-11, 3], [0, 4], [4, 3], [-11, 3], [-5, 4], [10, 3], [8, 2], [-7, 1], [-14, 2], [-20, 3], [-22, 4], [-7, 3], [-17, 4], [-17, 4], [-14, 3], [-29, 4], [-44, 4], [-35, 3], [-20, 4], [-35, 4], [-48, 4], [-46, 4], [-42, 4], [-27, 4], [-15, 4], [-19, 3], [-22, 4], [-25, 4], [-25, 3], [-19, 3], [-26, 3], [-24, 4], [-34, 4], [-49, 4], [-34, 3], [-49, 4], [-34, 3], [-49, 4], [-42, 4], [-33, 4], [-41, 3], [-42, 3], [-41, 2], [-26, 3], [-11, 2], [4, 1], [-11, 0], [-1, 1], [-15, 0], [0, 1], [0, 0]] (<class 'list'>)
ship :> inTheAir --> False (<class 'numpy.bool_'>)
ship :> p0 --> [4066.25684777   69.90764574] (<class 'numpy.ndarray'>)
trace --> [array([2500., 2698.]), array([2500., 2693.]), array([2500., 2686.]), array([2500., 2676.]), array([2500., 2664.]), array([2500., 2648.]),

In [26]:
# Run once for initial state 
step = 0 
state0 = State(
    step   = log[step]['step'],
    x      = log[step]['x'],
    y      = log[step]['y'],
    hspeed = log[step]['hspeed'],
    vspeed = log[step]['vspeed'],
    fuel   = log[step]['fuel'],
    angle  = log[step]['angle'],
    power  = log[step]['power'])

%f log :> [0] -->
%f state0 -->
%f log :> [0].values() list -->
%f state0 :> __dict__.values() list -->    

NameError: name 'State' is not defined

In [16]:
from typing import List, Optional, Callable, Tuple, Dict, Union  # OneNote2021 #_python_ #_typing_
GENE = Tuple[int,int]
GENOME = List[Gene]  # Genome 是一個 list 裡面都是 int
STATE = Dict[str,Union[int,float]]  # x,y,hspeed,power,.. etc 

Population = List[Genome]  # Population 是一個 list 裡面都是 Genome
PopulateFunc = Callable[[], Population]  # PopulateFunc() input nothing, output Population
FitnessFunc = Callable[[Genome], int]  # FitnessFunc() input a Genome, output an int value 
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]  # SelectionFunc() input Population and FitnessFunc, output a tuple of two Genomes
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]  # 這裡宣告了 MutationFunc 用 =  
PrinterFunc = Callable[[Population, int, FitnessFunc], None]


In [6]:
Genome

typing.List[typing.Tuple[int, int]]

In [3]:
FitnessFunc

typing.Callable[[typing.List[int]], int]

In [17]:
ATTR

typing.Dict[str, typing.Union[int, float]]

In [18]:
STATE

typing.Dict[str, typing.Union[int, float]]

In [24]:
a : GENE = (1,2)

In [25]:
type(a)

tuple

In [9]:
import typing
dir(typing)

['AbstractSet',
 'Any',
 'AnyStr',
 'AsyncContextManager',
 'AsyncGenerator',
 'AsyncIterable',
 'AsyncIterator',
 'Awaitable',
 'BinaryIO',
 'ByteString',
 'CT_co',
 'Callable',
 'ChainMap',
 'ClassVar',
 'Collection',
 'Container',
 'ContextManager',
 'Coroutine',
 'Counter',
 'DefaultDict',
 'Deque',
 'Dict',
 'ForwardRef',
 'FrozenSet',
 'Generator',
 'Generic',
 'Hashable',
 'IO',
 'ItemsView',
 'Iterable',
 'Iterator',
 'KT',
 'KeysView',
 'List',
 'Mapping',
 'MappingView',
 'Match',
 'MethodDescriptorType',
 'MethodWrapperType',
 'MutableMapping',
 'MutableSequence',
 'MutableSet',
 'NamedTuple',
 'NamedTupleMeta',
 'NewType',
 'NoReturn',
 'Optional',
 'OrderedDict',
 'Pattern',
 'Reversible',
 'Sequence',
 'Set',
 'Sized',
 'SupportsAbs',
 'SupportsBytes',
 'SupportsComplex',
 'SupportsFloat',
 'SupportsInt',
 'SupportsRound',
 'T',
 'TYPE_CHECKING',
 'T_co',
 'T_contra',
 'Text',
 'TextIO',
 'Tuple',
 'Type',
 'TypeVar',
 'Union',
 'VT',
 'VT_co',
 'V_co',
 'ValuesView',
 'W

http://localhost:8888/lab/workspaces/auto-m/tree/OneDrive/%E6%96%87%E4%BB%B6/Jupyter%20Notebooks/I%20study%20Pandas.ipynb

In [ ]:
import pandas as pd

## 研究 pandas read data 的眾多方法

我看其中就是 pd.read_clipboard() 好用。先把資料抓進 clipboard 然後執行之即得。
研究 iterations 數的分布的目的是想藉以調整 Annealing Algorithm 的 hyper-parameter 

In [ ]:
data = pd.read_clipboard()
data

In [ ]:
%f data :> describe() ==>
%f data :> Data.median() --> 

In [ ]:
data = pd.read_clipboard()
data

In [ ]:
%f data :> describe() ==>
%f data :> Iterations.median() --> 

### 調出 vspeed -40.2 看這樣算不算通過？算。但 Y=99.5 還不算觸地！

2021/04/28 21:04 咱的模擬完全正確，但是他的觸地標準出乎意外： Y 高度「四捨五入之後」 小於地平線 100 所以 Y=99.72 還不算觸地！ 

    state --> {'step': 86, 'x': 0, 'y': 136.72, 'hspeed': 0, 'vspeed': -35.14, 'fuel': 216, 'angle': 0, 'power': 0, 'force': -3.711}

    state --> {'step': 87, 'x': 0, 'y': 99.72, 'hspeed': 0, 'vspeed': -38.85, 'fuel': 216, 'angle': 0, 'power': 0, 'force': -3.711} <== 我認為這時候就安全降落了，但還沒。

    state --> {'step': 88, 'x': 0, 'y': 59.00, 'hspeed': 0, 'vspeed': -42.56, 'fuel': 216, 'angle': 0, 'power': 0, 'force': -3.711} <== 他到這時才算觸地，已經超速了。

2021/04/28 21:18 想想他是對的，因為 四捨五入 之故也要考慮不能讓 100.49  算作觸地，故然。但是 vspeed -40.2 算通過。所以只有 Y 高度挑剔。
		2021/04/28 21:20 承上，cost_of_state() 要改寫。


In [167]:
# 調出 vspeed -40.2 看這樣算不算通過？
state = State(**state0.__dict__)  # init for ecah genome 
genes = []
for i in range(8):
    state.next(0,0);
    genes.append([0,0])
for i in range(9,99):
    state.next(0,1);
    genes.append([0,1])
for i in [[0,1],[0,-1],[0,-1],[0,0],[0,0],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1],]:
    state.next(*i);
    genes.append(i)
genome, state = simulate_of_genome(genes)
%f genome count --> 
%f genome_to_command_list :> (pop()) -->
%f state -->
%f state cost_of_state :> (pop()) -->

genome count --> 110 (<class 'int'>)
genome_to_command_list :> (pop()) --> [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 3], [0, 2], [0, 2], [0, 2], [0, 1], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]] (<class 'list'>)
state --> {'step': 110, 'x'

### 錄取 genome 供研究用

In [ ]:
%%f 
\ 模擬安全降落

' genome [if] [] to genome [else] [] value genome [then] 
' state [if] [else] {} value state [then]

state0 State :> (**pop().__dict__) to state

9 [for] state :: next(0,0) genome :: append([0,0]) [next] 
91 9 - [for] state :: next(0,1) genome :: append([0,1]) [next] 

( 測試無誤 ) genome simulate_of_genome :> (pop()) dup :> [1] --> ==>
.s        

In [16]:
# 這組 genome 的 cost 原來是 208 分，竟然比 成功降落的 322 還要低！
# 僅修正 cost_of_state() 的誤解，現在變成 828 分了。
genome = [[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 0], [0, -1], [0, -1], [0, 1], [0, 1], [0, 0], [0, 1], [0, 1], [0, -1], [0, 1], [0, -1], [0, 0], [0, -1], [0, 0], [0, 0], [0, 1], [0, 1], [0, 0], [0, 1], [0, 1], [0, -1], [0, 0], [0, -1], [0, 0], [0, 1], [0, 1], [0, -1], [0, 1], [0, -1], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, -1], [0, -1], [0, 1], [0, 1], [0, 0], [0, -1], [0, 0], [0, -1], [0, -1], [0, 0], [0, 0], [0, -1], [0, -1], [0, 0], [0, 1], [0, 0], [0, -1], [0, -1], [0, 1], [0, 0], [0, 0], [0, -1], [0, -1], [0, 1], [0, -1], [0, -1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, -1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]
_ , __ = simulate_of_genome(genome)
%f _ -->
%f __ -->
%f __ cost_of_state :> (pop()) -->


_ --> [[0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 0], [0, -1], [0, -1], [0, 1], [0, 1], [0, 0], [0, 1], [0, 1], [0, -1], [0, 1], [0, -1], [0, 0], [0, -1], [0, 0], [0, 0], [0, 1], [0, 1], [0, 0], [0, 1], [0, 1], [0, -1], [0, 0], [0, -1], [0, 0], [0, 1], [0, 1], [0, -1], [0, 1], [0, -1], [0, 0], [0, 1], [0, 0], [0, 0], [0, 0], [0, -1], [0, -1], [0, 1], [0, 1], [0, 0], [0, -1], [0, 0], [0, -1], [0, -1], [0, 0], [0, 0], [0, -1], [0, -1], [0, 0], [0, 1], [0, 0], [0, -1], [0, -1], [0, 1], [0, 0], [0, 0], [0, -1], [0, -1], [0, 1], [0, -1], [0, -1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1], [0, -1], [0, 1], [0, 1]] (<class 'list'>)
__ --> {'step': 75, 'x': 0, 'y': 74.81249999999878, 'hspeed': 0, 'vspeed': -86.32499999999995, 'fuel': 308, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)
__ cost_of_state :> (pop()) --> 828 (<class 'int'>)


In [29]:
a = [1,2,3,4,5]
a = a[:3]
a

[1, 2, 3]

In [30]:
a.append(a[-1])
a

[1, 2, 3, 3]

In [30]:
# 擾動【安全降落】用來測試 sa() 的容忍度，也就是【局部最佳】的範圍。
# 發現 'y': -1180.6079999999897 也 bingo <--- 有問題
%f bd*
random.seed(100)
_ = sample_genomes["安全降落"][:-11]
_.append([0,-1])
genome, iterations, cost, state = sa(_)
%f \ genome count --> ==>
%f iterations -->
%f cost -->
%f state -->
cost_of_state(state)

B i n g o !
iterations --> 1 (<class 'int'>)
cost --> 390 (<class 'int'>)
state --> {'step': 116, 'x': 0, 'y': -1180.6079999999897, 'hspeed': 0, 'vspeed': -40.475999999999836, 'fuel': 110, 'angle': 0, 'power': 4, 'force': 0.28900000000000015} (<class '__main__.State'>)


390

### 查看 genomes 之間的距離

跨 epochs 記錄這些成績相近的 current & neighbout 然後查看它們之間的距離。好奇：同樣或成績近似的 vectors 會有環形分布的情形嗎？ --> Genomes 之間的距離怎麼樣也都在 4 ~ 11 之間，看不出蛋黃區、蛋白區的趨勢，作罷。

In [24]:
# Get distance of two genomes where genomes are vectors  
def distance(g1:GENOME, g2:GENOME):
    def vector(genome): 
        a = [i[1] for i in genome]
        assert len(a)<=200, 'Input genome length too long {}'.format(len(a))
        for i in range(200-len(a)):
            a.append(0)
        return np.array(a)
    return np.linalg.norm(vector(g1)-vector(g2))

%f distance :> ([[0,0],[0,0]],[[0,1],[0,1]]) -->

distance :> ([[0,0],[0,0]],[[0,1],[0,1]]) --> 1.4142135623730951 (<class 'numpy.float64'>)


In [27]:
%f bd* \ _*_
%f epoch :> ([[0,1]]) constant r1
%f epoch :> ([[0,1]]) constant r2
%f epoch :> ([[0,1]]) constant r3
%f epoch :> ([[0,1]]) constant r4

In [37]:
%f epoch :> ([[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]) constant ok1
%f epoch :> ([[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1]]) constant ok2

In [44]:
%f epoch :> ([[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]) constant freefall

{'cost': 1318, 'delta': -48, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 1173, 'delta': -145, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 854, 'delta': -319, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 765, 'delta': -89, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 724, 'delta': -41, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 739, 'delta': 15, 'temperature': 9.0, 'exp': 0.19}
{'cost': 739, 'delta': 0, 'temperature': 9.0, 'exp': 1.0}
{'cost': 520, 'delta': -219, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 531, 'delta': 11, 'temperature': 9.0, 'exp': 0.29}
{'cost': 342, 'delta': -189, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 316, 'delta': -26, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 301, 'delta': -15, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 249, 'delta': -52, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 264, 'delta': 15, 'temperature': 9.0, 'exp': 0.19}
{'cost': 264, 'delta': 0, 'temperature': 9.0, 'exp': 1.0}
{'cost': 249, 'delta': -15, 'temperature': 9.0, 'exp': 'n/a'}
{'cost': 249, 'd

In [30]:
distance(peforth.dictate('r1').pop()[0],peforth.dictate('r2').pop()[0])

7.280109889280518

In [42]:
%f r1 :> [0]  r2 :> [0] distance :> (pop(1),pop(0)) -->
%f r1 :> [0]  r3 :> [0] distance :> (pop(1),pop(0)) -->
%f r1 :> [0]  r4 :> [0] distance :> (pop(1),pop(0)) -->
%f r2 :> [0]  r3 :> [0] distance :> (pop(1),pop(0)) -->
%f r2 :> [0]  r4 :> [0] distance :> (pop(1),pop(0)) -->
%f r3 :> [0]  r4 :> [0] distance :> (pop(1),pop(0)) -->
%f r1 :> [2] -->
%f r2 :> [2] -->
%f r3 :> [2] -->
%f r4 :> [2] -->
%f ok1 :> [2] -->
%f ok2 :> [2] -->

%f ok1 :> [0]  r1 :> [0] distance :> (pop(1),pop(0)) -->
%f ok1 :> [0]  r2 :> [0] distance :> (pop(1),pop(0)) -->
%f ok1 :> [0]  r3 :> [0] distance :> (pop(1),pop(0)) -->
%f ok1 :> [0]  r4 :> [0] distance :> (pop(1),pop(0)) -->

%f ok2 :> [0]  r1 :> [0] distance :> (pop(1),pop(0)) -->
%f ok2 :> [0]  r2 :> [0] distance :> (pop(1),pop(0)) -->
%f ok2 :> [0]  r3 :> [0] distance :> (pop(1),pop(0)) -->
%f ok2 :> [0]  r4 :> [0] distance :> (pop(1),pop(0)) -->
    
%f ok1 :> [0]  ok2 :> [0] distance :> (pop(1),pop(0)) -->
    

r1 :> [0]  r2 :> [0] distance :> (pop(1),pop(0)) --> 7.280109889280518 (<class 'numpy.float64'>)
r1 :> [0]  r3 :> [0] distance :> (pop(1),pop(0)) --> 8.18535277187245 (<class 'numpy.float64'>)
r1 :> [0]  r4 :> [0] distance :> (pop(1),pop(0)) --> 8.54400374531753 (<class 'numpy.float64'>)
r2 :> [0]  r3 :> [0] distance :> (pop(1),pop(0)) --> 8.717797887081348 (<class 'numpy.float64'>)
r2 :> [0]  r4 :> [0] distance :> (pop(1),pop(0)) --> 9.38083151964686 (<class 'numpy.float64'>)
r3 :> [0]  r4 :> [0] distance :> (pop(1),pop(0)) --> 10.295630140987 (<class 'numpy.float64'>)
r1 :> [2] --> 213 (<class 'int'>)
r2 :> [2] --> 257 (<class 'int'>)
r3 :> [2] --> 223 (<class 'int'>)
r4 :> [2] --> 305 (<class 'int'>)
ok1 :> [2] --> 206 (<class 'int'>)
ok2 :> [2] --> 186 (<class 'int'>)
ok1 :> [0]  r1 :> [0] distance :> (pop(1),pop(0)) --> 7.54983443527075 (<class 'numpy.float64'>)
ok1 :> [0]  r2 :> [0] distance :> (pop(1),pop(0)) --> 8.246211251235321 (<class 'numpy.float64'>)
ok1 :> [0]  r3 :> [0] 

In [45]:
%f py> [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]
%f py> [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1],[0,-1]]
%f ( 安全降落 vs 更成功 ) distance :> (pop(1),pop(0)) -->


( 安全降落 vs 更成功 ) distance :> (pop(1),pop(0)) --> 5.5677643628300215 (<class 'numpy.float64'>)


In [46]:
%f py> [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]]
%f py> [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
%f ( 安全降落 vs 自由落體 ) distance :> (pop(1),pop(0)) -->

( 安全降落 vs 自由落體 ) distance :> (pop(1),pop(0)) --> 9.055385138137417 (<class 'numpy.float64'>)


### SA tested with a given genome

In [ ]:
%f bd*
%f sample_genomes :> ["安全降落"]
# random.seed(100)
genome, iterations, cost, state = sa(peforth.pop())

%f \ genome count --> ==>
%f iterations -->
%f cost -->
%f state -->
cost_of_state(state)

In [ ]:
# 擾動【安全降落】用來測試 sa() 的容忍度，也就是【局部最佳】的範圍。
# 發現 'y': -1180.6079999999897 也 bingo <--- 有問題
%f bd*
# random.seed(0)
_ = copy.deepcopy(sample_genomes["安全降落"])[:-21]
_.append([0,-1])
genome, iterations, cost, state = sa(_)
%f genome count --> nip
%f iterations -->
%f cost -->
%f state -->
%f state cost_of_state :> (pop()) -->
%f \ genome genome_to_command_list :> (pop()) -->

In [ ]:
# Mars Lander Episode-1 for the web site
import sys
surface_n = int(input())  # the number of points used to draw the surface of Mars.
land = []
for i in range(surface_n):
    land.append([int(j) for j in input().split()])
print(f"{land}", file=sys.stderr)    
genome = [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 1], [0, 2], [0, 3], [0, 3], [0, 3], [0, 4], [0, 4], [0, 3], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4], [0, 4]]
round = 0
while True:
    x, y, h_speed, v_speed, fuel, rotate, power = [int(i) for i in input().split()]
    print(f"0 {str(genome[round][1])}")
    round += 1
# Result remains fuel 325 

In [ ]:
[[int(j) for j in input().split()] for i in range(int(input()))]

# Error vars
prev_error = 0
sum_error = 0

# PID controller constants
KP = 49
KI = 1
KD = 49
TARGET_SPEED = -40.0

while True:
    X, Y, HS, VS, F, R, P = [int(i) for i in input().split()]
    
    # PID controller
    error = TARGET_SPEED - VS
    sum_error += error
    prev_error = error
    cmd = KP * error + KI * sum_error + KD * (error - prev_error)

    print(0, int(min(max(cmd, 0), 4)))

# result 314 